<a href="https://colab.research.google.com/github/adil-1522/Pytorch/blob/main/pytorch_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [8]:
#tokenize

def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [9]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [18]:
#vocab
vocab={'<UNK>':0}

In [19]:
def build_vocab(row):
  tokenized_question=tokenize(row['question'])
  tokenized_answer=tokenize(row['answer'])

  merged_tokens = tokenized_question +tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token]=len(vocab)



In [20]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [21]:
len(vocab)

324

In [22]:
#cnvert words to numerical values
def text_to_indices(text,vocab):

  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [23]:
text_to_indices("What is campusx",vocab)

[1, 2, 0]

In [24]:
import torch
from torch.utils.data import Dataset,DataLoader

In [30]:
class QADataset(Dataset):

  def __init__(self,df,vocab):
    self.df = df
    self.vocab=vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    numerical_question=text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [31]:
dataset = QADataset(df,vocab)

In [32]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [34]:
for question,answer in dataloader:
  print(question,answer[0])

tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([121])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([36])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([65])
tensor([[ 10,  75, 208]]) tensor([209])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([85])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([91])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([287])
tensor([[ 42, 137,   2, 226,  12,   3, 227, 228]]) tensor([155])
tensor([[ 10,  75, 111]]) tensor([112])
tensor([[ 10, 308,   3, 309, 310]]) tensor([311])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([16])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([244])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[ 1, 

In [35]:
import torch.nn as nn

In [65]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn=nn.RNN(50,64,batch_first=True)
    self.fc=nn.Linear(64,vocab_size)

  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden,final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output

In [66]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [73]:
learning_rate = 0.001
epochs = 20

In [74]:
model = SimpleRNN(len(vocab))

In [75]:
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [76]:
for epoch in range(epochs):

  total_loss=0;

  for question,answer in dataloader:

    optimizer.zero_grad()

    output = model(question)

    loss = criterian(output,answer[0])

    loss.backward()

    optimizer.step()

    total_loss = total_loss+loss.item()

  print(f"Epoch: {1+epoch},Losss: {total_loss:4f}")

Epoch: 1,Losss: 522.267374
Epoch: 2,Losss: 451.961525
Epoch: 3,Losss: 373.393134
Epoch: 4,Losss: 309.772755
Epoch: 5,Losss: 257.947262
Epoch: 6,Losss: 209.225058
Epoch: 7,Losss: 166.912361
Epoch: 8,Losss: 129.792884
Epoch: 9,Losss: 99.828098
Epoch: 10,Losss: 76.865356
Epoch: 11,Losss: 59.264010
Epoch: 12,Losss: 46.502632
Epoch: 13,Losss: 36.975396
Epoch: 14,Losss: 30.093268
Epoch: 15,Losss: 25.054877
Epoch: 16,Losss: 20.438547
Epoch: 17,Losss: 17.432324
Epoch: 18,Losss: 14.640722
Epoch: 19,Losss: 12.488488
Epoch: 20,Losss: 10.948924


In [87]:
def predict(model,question,threshold=0.5):

  numerical_question = text_to_indices(question,vocab)

  question_tensor = torch.tensor(numerical_question).unsqueeze(0)


  output = model(question_tensor)

  probs = torch.nn.functional.softmax(output,dim=1)

  value,index = torch.max(probs,dim=1)

  if value < threshold:
    print("I dont know")

  print(list(vocab.keys())[index])

In [88]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [94]:
list(vocab.keys())[7]

'paris'